In [56]:
import numpy as np

In [ ]:
"MDP"

class Gato:
    def __init__(self):
        self.estados = self.gen_estados()
        self.acciones = [(i, j) for i in range(3) for j in range(3)]
        self.gamma = 1  # factor de descuento
        self.valores = {estado: 0 for estado in self.estados}  # para la iteración de valor 
        self.politica = {estado: None for estado in self.estados}  # para la iteración de política

    def gen_estados(self):
        def _todos_estados():
            all_juego_config = set()
            for valores in np.ndindex(3, 3, 3, 3, 3, 3, 3, 3, 3): # 3 valores que representan: X,O,' '
                estado = tuple(valores)
                all_juego_config.add(estado)
            return all_juego_config

        def ganar_2(estado): # verifica aquellos estados en los que se tengan dos ganadores 
            count = 0
            for i in range(3):
                if estado[i*3] == estado[i*3+1] == estado[i*3+2] and estado[i*3] in [1, 2]:
                    count += 1
                if estado[i] == estado[i+3] == estado[i+6] and estado[i] in [1, 2]:
                    count += 1
            if estado[0] == estado[4] == estado[8] and estado[0] in [1, 2]:
                count += 1
            if estado[2] == estado[4] == estado[6] and estado[2] in [1, 2]:
                count += 1
            return count >= 2

        estados = _todos_estados()
        valid_estados = set() # verifica aquellos estados que sí pertenecen al juego
        for estado in estados:
            x_count = estado.count(1)
            o_count = estado.count(2)
            if abs(x_count - o_count) > 1: # verificar que la diferencia de turnos no sea mayor a 1
                continue
            if ganar_2(estado):
                continue
            if x_count > o_count:
                continue  # Eliminar los turnos del jugador aleatorio
            valid_estados.add(estado)
        return valid_estados

    def terminal(self, estado): # definición de los estados terminales
        return self.ganador(estado, 1) or self.ganador(estado, 2) or 0 not in estado

    def ganador(self, estado, jugador): # verificar los estados en los cuales ya se tiene un ganador
        juego = np.array(estado).reshape(3, 3)
        for i in range(3):
            if all(juego[i, :] == jugador) or all(juego[:, i] == jugador):
                return True
        if all(juego.diagonal() == jugador) or all(np.fliplr(juego).diagonal() == jugador):
            return True
        return False

    def recompensa(self, estado): # se define la recompensa, 1 si gana, -1 si pierde, 0 si se tiene un empate
        if self.ganador(estado, 1):
            return 1
        if self.ganador(estado, 2):
            return -1
        return 0
    
    "Iteración de valor"

    def valor(self, epsilon=1e-6): 
        while True:
            self.iteracion_politica(epsilon)
            stable = True
            for estado in self.estados:
                if self.terminal(estado):
                    continue
                old_accion = self.politica[estado]
                best_accion = None
                best_value = float('-inf')
                for accion in self.acciones:
                    if estado[accion[0] * 3 + accion[1]] == 0:
                        q_val = self.q_value(estado, accion)
                        if q_val > best_value:
                            best_value = q_val
                            best_accion = accion
                self.politica[estado] = best_accion  # mejorar el valor respecto a la política
                self.valores[estado] = best_value  
                if old_accion != best_accion:
                    stable = False
            if stable:
                break


    "Iteración de política"
    def iteracion_politica(self, epsilon=1e-6): 
        while True:
            delta = 0
            for estado in self.estados:
                if self.terminal(estado):
                    self.valores[estado] = self.recompensa(estado)
                    continue
                v = self.valores[estado]
                accion = self.politica[estado]
                if accion is not None:
                    self.valores[estado] = self.q_value(estado, accion)
                delta = max(delta, abs(v - self.valores[estado]))
            if delta < epsilon:
                break

    def q_value(self, estado, accion):
        if estado[accion[0] * 3 + accion[1]] != 0:
            return -np.inf
        new_estado = list(estado)
        new_estado[accion[0] * 3 + accion[1]] = 1
        return self.recompensa(new_estado) + self.gamma * self.valores.get(tuple(new_estado), 0)



In [58]:
mdp = Gato()
mdp.valor()
mdp.iteracion_politica()

In [59]:
print("\nSalida:")
for estado, valor in sorted(mdp.valores.items(), key=lambda x: x[1], reverse=True):
    best_accion = mdp.politica[estado]
    print("Estado:", estado,"Valor:", valor, "Política óptima:", best_accion if best_accion else "No hay acción")


Salida:
Estado: (2, 0, 0, 2, 2, 0, 1, 1, 0) Valor: 2 Política óptima: (2, 2)
Estado: (0, 2, 1, 2, 1, 1, 0, 2, 2) Valor: 2 Política óptima: (2, 0)
Estado: (2, 1, 2, 0, 1, 0, 0, 0, 2) Valor: 2 Política óptima: (2, 1)
Estado: (0, 2, 0, 0, 1, 2, 1, 0, 2) Valor: 2 Política óptima: (0, 2)
Estado: (2, 2, 0, 1, 0, 1, 1, 2, 2) Valor: 2 Política óptima: (1, 1)
Estado: (0, 1, 2, 0, 1, 2, 2, 0, 0) Valor: 2 Política óptima: (2, 1)
Estado: (0, 0, 1, 2, 1, 0, 0, 2, 2) Valor: 2 Política óptima: (2, 0)
Estado: (2, 2, 0, 1, 0, 1, 0, 0, 2) Valor: 2 Política óptima: (1, 1)
Estado: (0, 0, 2, 2, 1, 1, 2, 1, 2) Valor: 2 Política óptima: (0, 1)
Estado: (1, 0, 1, 2, 1, 2, 2, 0, 2) Valor: 2 Política óptima: (0, 1)
Estado: (2, 0, 0, 0, 2, 2, 1, 0, 1) Valor: 2 Política óptima: (2, 1)
Estado: (0, 0, 1, 2, 1, 2, 2, 1, 2) Valor: 2 Política óptima: (0, 1)
Estado: (0, 2, 0, 2, 1, 2, 1, 0, 0) Valor: 2 Política óptima: (0, 2)
Estado: (0, 0, 1, 1, 2, 2, 1, 2, 2) Valor: 2 Política óptima: (0, 0)
Estado: (1, 0, 2, 0, 1, 2